# Regresión Lineal

#### Importamos la libreria scikit-learn

In [3]:
import sklearn.linear_model as lm
import sklearn.model_selection as ms

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

### Analisis de los datos

In [4]:
df = pd.read_csv('regression_data.csv')
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,7129300520,10/13/2014,3,1.00,1180,5650,1.0,0,0,3,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,221900
1,6414100192,12/9/2014,3,2.25,2570,7242,2.0,0,0,3,...,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,538000
2,5631500400,2/25/2015,2,1.00,770,10000,1.0,0,0,3,...,770,0,1933,0,98028,47.7379,-122.233,2720,8062,180000
3,2487200875,12/9/2014,4,3.00,1960,5000,1.0,0,0,5,...,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,604000
4,1954400510,2/18/2015,3,2.00,1680,8080,1.0,0,0,3,...,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,510000


##### Regresion lineal de los datos

In [7]:


X = df['x'].values.reshape(-1, 1)
y = df['y'].values.reshape(-1, 1)

KeyError: 'x'

##### Dividimos entre Train y Test

In [6]:
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=0)

NameError: name 'X' is not defined